# 【確認課題】Pandas

このPandas確認課題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [1]:
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [3]:
columns = df_receipt.loc[:,["sales_ymd","customer_id","product_cd","quantity","amount"]]
mask = (columns["customer_id"] == "CS018205000001") &((columns["amount"] >= 1000 ) | (columns["quantity"] >= 5))
print (columns[mask].head())

       sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [4]:
print (df_customer.sort_values(by="birth_day",ascending = False).head(5))

          customer_id customer_name  gender_cd gender   birth_day  age  \
15639  CS035114000004         大村 美里          1     女性  2007-11-25   11   
7468   CS022103000002        福山 はじめ          9     不明  2007-10-02   11   
10745  CS002113000009        柴田 真悠子          1     女性  2007-09-17   11   
19811  CS004115000014         松井 京子          1     女性  2007-08-09   11   
7039   CS002114000010          山内 遥          1     女性  2007-06-03   11   

      postal_cd                address application_store_cd  application_date  \
15639  156-0053     東京都世田谷区桜**********               S13035          20150619   
7468   249-0006    神奈川県逗子市逗子**********               S14022          20160909   
10745  184-0014  東京都小金井市貫井南町**********               S13002          20160304   
19811  165-0031    東京都中野区上鷺宮**********               S13004          20161120   
7039   184-0015  東京都小金井市貫井北町**********               S13002          20160920   

          status_cd  
15639  6-20091205-6  
7468   0-00000000-0  
10

---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [5]:
print (len(df_receipt))

104681


## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [6]:
unique_count = df_receipt["customer_id"].nunique()
print (unique_count)

8307


<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [7]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [8]:
amount_per_customer = df_receipt_only_member.groupby("customer_id")["amount"].sum()
print(amount_per_customer[amount_per_customer.mean() <= amount_per_customer].head())

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
Name: amount, dtype: int64


<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [9]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [10]:
df_female_member = df_customer_only_member.query("gender_cd == 1")
profit_sum_only_member = df_receipt.groupby("customer_id")["amount"].sum()
merge_genj_profitsum= pd.merge(df_female_member,profit_sum_only_member,on="customer_id",how="outer")
print(merge_genj_profitsum.fillna(0).head())

      customer_id customer_name  gender_cd gender   birth_day   age postal_cd  \
0  CS021313000114        大野 あや子        1.0     女性  1981-04-29  37.0  259-1113   
1  CS031415000172       宇多田 貴美子        1.0     女性  1976-10-04  42.0  151-0053   
2  CS028811000001        堀井 かおり        1.0     女性  1933-03-27  86.0  245-0016   
3  CS001215000145         田崎 美紀        1.0     女性  1995-03-29  24.0  144-0055   
4  CS015414000103         奥野 陽子        1.0     女性  1977-08-09  41.0  136-0073   

                  address application_store_cd  application_date  \
0    神奈川県伊勢原市粟窪**********               S14021        20150905.0   
1     東京都渋谷区代々木**********               S13031        20150529.0   
2  神奈川県横浜市泉区和泉町**********               S14028        20160115.0   
3     東京都大田区仲六郷**********               S13001        20170605.0   
4      東京都江東区北砂**********               S13015        20150722.0   

      status_cd  amount  
0  0-00000000-0     0.0  
1  D-20100325-C  5088.0  
2  0-00000000-0     0.0  


<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [11]:
df_customer["application_date"] = pd.to_datetime(df_customer["application_date"],format= "%Y%m%d")
print(df_customer[["customer_id","application_date"]].head())

      customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [12]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
sum_only_member = df_receipt_only_member.groupby("customer_id").sum()
sum_only_member["log_amount"] = np.log10(sum_only_member["amount"])
print(sum_only_member[["log_amount","amount"]].head())

                log_amount  amount
customer_id                       
CS001113000004    3.113275    1298
CS001114000005    2.796574     626
CS001115000010    3.483445    3044
CS001205000004    3.298416    1988
CS001205000006    3.523356    3337


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [13]:
print(df_product.isnull().sum())

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64


---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [14]:
df_product_1 = df_product.dropna()

In [15]:
print(len(df_product), len(df_product_1))

10030 10023


---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [16]:
mean = np.round(df_product.mean(numeric_only=True))
df_product_2 = df_product_1.fillna(mean)
print(df_product_2.isnull().sum())

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64


### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。

![BingAIの回答例](https://github.com/shinonome-inc/PythonTutorialForDSCourse/blob/main/exercise/images/BingAI.png?raw=1)